In [1]:
import optuna
from optuna.integration import TFKerasPruningCallback
import tensorflow as tf
import numpy as np
import healpy as hp
import os
import matplotlib.pyplot as plt
from deepsphere import HealpyGCNN, healpy_layers as hp_layer
from deepsphere import utils
from healpy import read_map

data_directory = "/mnt/lustre/scratch/nlsas/home/csic/eoy/ioj/CMBFeatureNet/data/"
os.chdir(data_directory)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #disable GPU
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  #suppress TF warnings
print("Current working directory:", os.getcwd())

/mnt/lustre/scratch/nlsas/home/csic/eoy/ioj/SkyNeuralNets/cmbnet_env2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-13 22:58:34.565400: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 22:58:34.862263: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 22:58:39.058825: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 22:58:41.894481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to regist

KeyboardInterrupt: 

In [ ]:
from astropy.io import fits
def read_map(file_path):
    """
    Reads a Healpy map from a FITS file and flattens the data.
    """
    
    with fits.open(file_path) as hdul:
        hdul.info()
        if len(hdul) > 1 and hasattr(hdul[1], 'columns'):
            print(hdul[1].columns)
        return np.concatenate(hdul[1].data['T'])

In [ ]:
def read_all_maps(path_lcdm, path_feature, n_maps=100):
    maps = []
    labels = []
    
    #LCDM maps
    for i in range(n_maps):
        map_lcdm = read_map(f"{path_lcdm}cmb_map_{i}.fits")
        maps.append(map_lcdm)
        labels.append(0)  #lcdm
    
    #Feature maps
    for i in range(n_maps):
        map_feature = read_map(f"{path_feature}cmb_map_feature_{i}.fits")
        maps.append(map_feature)
        labels.append(1)  #feature
    
    maps = np.array(maps).astype(np.float32)[..., None]  #Add channel dimension
    labels = np.array(labels).astype(np.int32)
    #print(labels)
    return maps, labels

In [ ]:
#Read the data
path_lcdm = "./simulated_maps/lcdm/"
indices = np.arange(hp.nside2npix(nside))
path_feature = "./simulated_maps/feature/"
x_raw, y_raw = read_all_maps(path_lcdm, path_feature, n_maps=36) #0: lcdm, 1:feature

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_raw, y_raw, test_size=0.3, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
def objective(trial):
    tf.keras.backend.clear_session()

    #Hyperparameters to search
    num_blocks = trial.suggest_int("num_blocks", 2, 5)
    K = trial.suggest_categorical("K", [3, 5, 7, 10])
    base_Fout = trial.suggest_categorical("base_Fout", [4, 8, 16])
    activation = trial.suggest_categorical("activation", ["relu", "leaky_relu"])
    use_bn = trial.suggest_categorical("use_bn", [True, False])
    use_dropout = trial.suggest_categorical("use_dropout", [True, False])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)

    #Build the model layers
    layers = []
    Fout = base_Fout
    for i in range(num_blocks):
        layers.append(
            HealpyChebyshev(K=K, Fout=Fout, use_bias=True, use_bn=use_bn, activation=activation)
        )
        if use_dropout:
            layers.append(tf.keras.layers.Dropout(dropout_rate))
        layers.append(HealpyPool(p=1))
        Fout = max(2, Fout // 2)  #optional: decrease Fout gradually

    layers.append(HealpyChebyshev(K=K, Fout=2))
    layers.append(tf.keras.layers.Lambda(lambda x: tf.nn.softmax(tf.reduce_mean(x, axis=1), axis=-1)))

    model = HealpyGCNN(nside=nside, indices=indices, layers=layers, n_neighbors=20)
    model.build(input_shape=(None, len(indices), 1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train
    history = model.fit(
        x_train, y_train,
        batch_size=16,
        epochs=50,
        validation_data=(x_val, y_val),
        verbose=0,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
        ]
    )

    val_acc = max(history.history['val_accuracy'])
    return val_acc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print(f"  Params: {trial.params}")